<a href="https://colab.research.google.com/github/ThomasMiconi/MetaMetaLearning/blob/main/MML_DMS_BS500_400T_Simplify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Feb  8 19:55:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------


Testing with small params to test the graphing ! Seems OK, tried DMS. 500 gen per run, 6 runs. Died at gem 373 of run 5,. But seems to work for these 5 runs!

Now trying no plasticity:


===

Trying to simplify, e.g. using 1-hot responses instead of scatter, remove some commented lines, etc.

Using BS=500 for test. 29s per trial, i.o. 39s/trial with BS=1K.

===


Trying same but different sequences of stimuli across batch (but same for antithetic pairs) (Also: 1401 gens exactly). OK.

Trying 400 trials for DMS io 600: seems like it might work in 1400 gens, and training might look better initially. Need more data.



Trying InitAlpha 0, 600 trials, NOT preventing negative alphas (I need to think if that's feasible under antithetic mutations).... 

But wait !!!  If I do that, plasticity will necessarily increase, and then Reviewer 3 will say "well, duh, yeah, it's better after evolution, but that's just bvecause evolution pumped up the plasticity - it just flipped up a single-bit 'on' switch!"

Maybe better to try IA .25, 400 trials. Or .5, 300 trials even. 

.5 300 trials: learns, but not enough, 2 runs reach ~ .85 and .7 after 24 hours (~2200 gens)

400 trials. Reaches ~ .8+ after 1600 gens.

Trying exact same but with mutation 3x for alpha. Not sure if better... Will try another pair of runs, this time with all (non mutated) batch individuals for the 1-block runs on testing... Might have been good but I couldnt' get the late files !

Trying 300 trials, 1.0 alphainit? No, too good on gen 1.

Back to 400 .5, still 3x on Alpha Mutations:

Very high correlation between all the individuals'scores at a given trial ?? How ?

Started ~5pm, killed ~3pm. Trying Colab pro+.

With v100, each gen takes 29s i.o. 51s! But I want to try independent perturbation for each element in batch, which now takes 57s... 49s if I do the same fluctuations for both antithetic pairs.... Let's try this.

Seems quite fluctuating / unstable! Perf not bad... Though WH crashes a bit at the end!  Need to try without the 3x for alpha... Doing that: Perf seems less ! 

Need to try: common perturbation, with either more time or largwr BS, exploiting the V100...

Back to common perturbation, still removed the 3x for alpha mutations. 33s.

BS=1000: 40s. 2000: 65s. Let's go with BS=1000. Doesn't seem to improve much... Let's try again with same.




---


OK, previous seems to work-ish, slowly, but now the training tasks have very high initial performance...

I need to try 1 task per gen, then 1 task per gen with lower LR, then with alphaplast .5, then when I choose the best version I should run it with alphainit back to 0.5.

First try 1 task per gen (still infinite gen), after some significant code clean and ensuring that I save the 1st and last block, both on test task: Doesn;t help, below previous.

Same but with an additional .3x on the LR: Also seems slower.

OK, remove .3x, 2 tasks per gen, alphainit .5... Need to check initial perf of learned.


(Should try reducing MaxDW !! Also with different stimuli over the population)

Works similarly to AI 1.0, though it may plateau earlier?  Hits max at ~ .88, doesn't break up...


---

Trying WH task = dms, 500 trials, 800 gens: At about 720 gens, not great. Some learning, but.

Trying with alphainit 1.0. (Should also try 3 tasks per gen.)

Similar, it seems to be going up still, but with much irregularity and variance.... What if I try for 1000 gens? But the 680 gens are 13h already !

3 tasks per gen, 400 trials per task, Inf gens. (59 seconds per 3-task gen on a P100...) After 800 gens, one run shows little learning at all, the other initially learnt somewhat but then fell back... Basically seems much worse than before.

Will try 600 trials per task, 2 tasks per gen, Inf gens.

960 gens: Is the median of 2 runs plateauing a bit belo .8? Still a lot of irregularity - even with the same genomes, median varies enormously! e.g. Med task mseloss: 0.2744 followed by Med task mseloss: 0.8297 ! (How is that even possible over 250 different genomes each with completely different input sequences?)

No, they have all the exact same input sequence !

Genuinely not clear which one should work better... Need to try both.

Mean of the 2 runs hits .83, then later .87. 4 runs seems to confirm...


---

Probably just go back to 200 gens, .5, full LR (no .5), alphainit, see what happens with enough runs.

OK but some learning for gen 1...

250 trials, 400 gens: not much learning at all
Same, alphainit 1: back to some learning for gen 1. (And gen400 not so good)

350 trials, alphaint .5, 300 gens: Hm. Gen 0 has no visible learning. But, seems to be not great at gen 300, ~80%. Not much learning after 200 gens anyway.

Also seems to have more noise than 500 trials. 

Should try 400 trials? (And maybe reduce the LR?)

Should also look at clipping alphas at 0

---


OK, now putting more runs, 200 gens, fixing a few things to get ready for figures. (I wang to get the actual blosses for each trial for lifetimes 1 and NbGen).


Note that there is a little bit of lifetime learning even at generation 0.
At generation 200, last 100 trials are ~95% correct. 

In the first 4 runs, in gen 0, there is a clear initial bias towards wrong answers for NAND. Possibly just noise. It makes the evolutionary learning look like you're just elevating the whole-lifetime learning by a fixed amount...


Trying just to see the initial with ALPHAINIT .25: Initial (gen 0) looks to have much less learning. Does the evolution still work though?

Let's try it!

Learns, but by 200gens it's about 90% correct....

All same, but 100+400 i.o. 400+100. Works much less well! (doesn't seem to be jiust because it's harder overall, the learning really seems quite noisy... if there's any !)

Back to 400+100...

Note that the same noise vector is applied to all batch elements, but all batch elements have randomly different stimulus order...

Giving exact same stimuli to all individuals (could also try just giving same stimuli to antithetic pairs) :

Proportion correct in the median of withheld reaches ~83% correct. Not sure if better or worse.

Trying same stimuli only for antithetic, and BS 500 -> 1000: Out of memory ! Back to 500: Not much difference.

BS 1000: time goes from 67s to 111s... Might be worse initially, but not sure near the end... But still around 85%...

Probably better to try 300 gens...? Or increase LR? ... 


Although mean Alpha slightly goes down, alpha has a lot of variation! Mean max med min:
tensor(0.23811, device='cuda:0') tensor(0.62160, device='cuda:0') tensor(0.23806, device='cuda:0') tensor(-0.26484, device='cuda:0')


Trying BS=500, 300 gens. Doesn't improve much after 200 gens. Very noisy? What if reducing LR ? Or 3 tasks per gen? 

LR x.2 (note it doesn't have the .5): Very difficult to see differences.....

Lots of variation. 4 runs probably not enough.

---


Back to nand. 

Not very nice, probably bvecause 1 task per gen: lots of spikes in the training tasks, improvement on nand is small (because it's very easy!)

Should try 2 tasks per gen and lower initial alphainit? 

2 tasks per gen, alphainit 0 ! Doesn 't learn anything, even if I push mutationsize to .05 io .01...

Back to mutsize .01, initalpha 1.0. It works, but initial withheld binary perf already very high, and mseloss is trending well below the learned tasks...

Same with initalpha .5? Also, I know make the very first generation a WH task.


---

Alphainit 1.0, 1 task per gen, 1500 gens, LR .5x: 

Still, some learning, but irregular, much variance especially at the end (oscillating wildly). 

Same but lr and wdecay / 10 at each 1/3 of the 1500 gens.
Not much diff. W changes are very low in the second regime, almost negligible in 3rd, e.g.:

norm w: 14.5615 norm gradient: 0.7282
Norm w-wprev: 0.0006 Max abs w-wprev: 0.0000

Should try same but start from 10x LR ?... Now nothing learns.

TO TRY: Use more tasks per gen, e.g. by stacking tasks within a single batch.   Reduce stimsize to 1.   Ensure exact same stimuli for both antithetics ?

---

Exact same (Jinit .5, LR normal,1/3 WDecay) but not ranked losses : 

Quite similar to thhe previous one... WH goes a bit up first, then down....

So the ranked losses didn't make much difference. (It wasn't applied to alpha anyway!)

Norm w quite low, ~13.5. Not stabilized.

JInit 1.5, wdecay 3x, lr 2x: Unstable learning for WH...

Jinit 1.5, wdecay 1/3x, lr normal: There is learning, but WH is still quite irregular, same-net variance .2-.7 !

Note: There are no big outliers in lifelosses, even for WH. Not clear that trimming the extremities would help.



---



Exact same except tanh and Jinit 1.5. Should I re-double the evo LR ?... Not for now...

It seems to kind of work, there is clearly some WH learning in both runs, but real noisy (for the WH), especially at the end... And it takes ~11 hours !

LEarned tasks are a bit less noisy nbu tstill ave noise, especially at the end too.

Even for learned tasks there is also noise in the network results, e.g. Med/min/max/Half-Nth/0th loss in batch: 0.1629411280155182 0.10090695321559906 0.6069034337997437 0.1539154201745987 0.5359363555908203

And that's despite the fact that learned gens have two different tasks... OTOH WH gens have two blocks as well, just of the same task...

Trying 2x LR: Learning still quite noisy, especially for WH. Still much same-net variance.

MaxDW 1e-2 -> 3e-3: Certainly doesn't prevent huge same-net variation specifically for non-trained.
Med/min/max/Half-Nth/0th loss in batch: 0.2966817021369934 0.16374562680721283 0.9815937876701355 0.24325412511825562 0.8848958015441895

One possibility is that certain patterns specifically force the network into a specific learned task, which is OK for the learned ones and very bad for the unlearned.

MaxDw back to 1e-2. Jinit .5, and alpha also started from the same type of init, very low (but positive, unlike w). Vry little / no learning, even for the training tasks, in 600 gens !

Need to try 1- putting back alpha init to .5 and 2- rank scores (normalized 0-1 then /5)

Trying exact same but alpha back to initial Alphaint (i.e. .5)": Now one learns WH OK (after a no good phase), other only learns a bit, very irregular. Training tasks learned OK, though with spikes.

W norm grows massively compared to init state, alpha decreases to <.4? Std W: 0.31415012 Norm W 21.992723
Std w_init_like: 0.059975140018897144 norm: 4.198762373075644


Will need to try large weight decay...

For now, exact same but with ranked losses:
norm w-wprev should go to ~.2... does? Maybe a bit lower, -.15? 
LEarning seems less good, certainly for unseen. Maybe a bit for trained.

Try with doubled (again) LR ! Well, 2.5x: Goes down fast initially but very irregular, irregularity increases, even for training tasks. Much higher weights, norm ~40.

Trying LR back to previous, wdecay 2.5x: Learning OK (not diff from usual?) on trained, bad on WH. weights not that much lower at the end, norm ~15 io ~20...

Now trying LR 2.5x again: bad, even for training tasks, very high norm w (38)

Now LR without even the 2x, and wdecay .333x: 

Trained learn, though not very well. WH doesn't learn at all!

Seems that ranked losses was a bad idea.


---

No 20%. Alpha whole matrix. LR back to .5x. 780 gens. JInit 1.0, alphainit still .5. tau_et 800->1000.  Keep l_relu for now but consider going back to tanh....


So tantalizing! One does seem to learn... The other may start learning a bit near the end!

Both have still pretty bad learning for some of the training task... Again antiwatchstim1 is pretty bad. Worse thqtn withheld in one!

Still massive variance in withheld: Med/min/max/Half-Nth/0th loss in batch: 0.7925445437431335 0.5809413194656372 2.91385817527771 0.8041354417800903 0.6860721111297607. But not so much on trained tasks? Still large, 2x range.

Mean/median/max plastic weights (withheld): 0.008706522, 0.0053214924. 0.264261



---

Shifted DT and TAU and TAU_ET to real values - 20, 1000 and 800 (currently). The time periods for the stimuli are still in time steps. It's OK.

Plasticity starting from trial 40 i.o. 100.

OK, works... (can be pretty noisy!) bl4/ml4.

Trying with Alpha a column vector. Maybe better, not sure, would need more runs.

Trying WH dms (and dnms), with 20% more 10/01/anti10/anti01 - in the hope that it will build appropriate representations to allow for withheld dms to be learned.

Doesn't work at all, no dms learning though all other ones are learnt well !

Of course, if you had separate representations for NOR and AND, that would suffice to get dms...

Should try with maxdw 1e-2 , eta .1, 400 gens.

Trying with maxdw 1e-2, eta still .03, 400 gens: One runs learns WH initially - all the way to ~gen 250! - then unlearns it. Other never really seems to, at least for element 0... 

Variance is enormous on test trials, e.g. (last gen - note diff b/w 0 and 250): 
Med/min/max/Half-Nth/0th loss in batch: 0.46544599533081055 0.2260051965713501 1.1365395784378052 0.43774789571762085 0.6898736357688904

Maybe tune down the chaos initialization?...


Should also try with softplus / l_relu i.o. tanh?...

Trying l_relu, clipping at 5, JINIT 1.5. Not sure. Very noisy evolution, even for the trained ! 

Also maybe use rank scores to prevent wild variance? But note that often for training gens the variance is much lower, e.g. Med/min/max/Half-Nth/0th loss in batch: 0.6700206995010376 0.5432029962539673 0.8137558698654175 0.6717157363891602 0.698481023311615


Other problem: because input neurons have largest activations, they crete the largest plastic input weights in other neurons, due to plasticity being per-neuron...

Should get rid of the 20%...



---

Trying exact same but with alphainit .5 -> 1.0: Same, doesn't really work for withheld. Less troubled with antiwatchstim1?

It doesn't work with dms/dnms because the system does not develop the relevant representations for in-life learning to exploit usefully. But all we need is separate "no firing" and "both firing" indicators! How could we favor the emergence of these?

Alphainit back to .5, Evo LR 2x (so back to before plast). Same, doesn't work.


Need to try same with nand instead of dms. Then if not working, reduce back Evo LR. If that again doesn't work, more trials  and generations.

OK, works. Binary a bit ugly though.

Short term: 
- Make the 'nominal' dt larger without changing the dynamics - check the tau constants
- Plasticity only starts after numtrial 100? What happens if earlier?

Medium term:
- Put the neuromodulation under network control, rather than just using the raw error signal.
- Find way to improve the binary (though maybe you just need more time)
- If possible, find a way to encourage emergence of more complex representations, i.e. indicators for 'firing' and 'no firing' that would allow for learning dms/dnms. 




---


500 trials/gen, 475 gens, withhheld task is dms.

Worse, not absolutely clear there is any learning on withheld.

In both runs, antiwatchstim1 (but not watchstim1) is also very bad, even a bit worse than withheld dms...


Trying with 2 tasks per gen. 240 gens. (Next thing to try would be same but 2x LR)

If DMS too hard, maybe use nand ?

Seems bad at antiwatchstim1, better at watchstim1. Other antis don't seem to make such a difference? Even when it's quite good at antiwatchstim1 at initialization, it becomes worse at it over training??

Really good at nand, even though not at initialization - it just meta-learns nand really well.

Performance on withheld goes downward at some point.... It's not a matter of not evolving fast enough, we should prioritize in-life learning over evo learning more... Put back alphainit = 1.0 ? 

One of the two runs pushes withheld loss very high, much worse than init! Actively anti-learns it.

On the learned tasks,  within-batch variance can be very high
e.g.: Min/max/med loss in batch: 0.2571086883544922 0.6232863664627075 0.35362815856933594
On the withheld task, within-batch variance is enormous in the run that learned it somehow (e.g. .30-.80, median .5), and not big in the other one (everything is terrible).

Basically (bl4bad.txt): one of the runs may or may not be learning withheld, the other definitely doesn't. They both seem to be learning the training tasks, though binary loss is bad.



---


Added 01/10/anti01/anti10 tasks. InitAlpha .25 -> .5.  900 trials / task (still 1 task per gen). Now withheld task is 'and', and I also remove nand.

(Might think of checking maxdw ? 1e-3 -> 1e-2 ?)

Withheld is initially a bit better than the others, but at the end the others are a bit better (on average).

It starts from pretty high too....

OK, now seems to work ! But maybe actually works too fast, lol? In one run, withheld hits very high within 80 gens. Though there are many downstrokes later! But eventually it stabilizes high (after ~400 gens).

w and alpha show no obvious structure. Overall alpha level has not change much from ~.5, if anything slightly lower.
Plastic weights show a lot of structure! Squarish, i.e. both vertical and horizontal structure, though horiz stripes seems to dominate? Not clear how much of the structure is shared among the various batch elements...

(Should try with randomly-swapped alphas, lol)


To try:   
- 500 trials per gen
- dms withheld task instead of and 
- test final network with randomly swapped alpha.


---

Adding tasks or, nor, and, nand. 

Now training tasks include neither dms nor dnms.

Still ~1000 gens, 200 trials per task, 1 task per gen. No visible learning at all.

500 trials per task? 600 gens.

May of may not show a bit of learning in the withheld task (the others are learning, a bit, it seems). One run seems to learn a little bit, other less? 

b;4reft.txt saved the two runs.

- more tasks. Add 1/0 only, 0/1 only, and their anti.
- Two tasks per gen
- Longer tasks (more trials)
- Cycle through all tasks in regular order, just don't perform the gradient step if task == withheld
- choose a different test task (and?)
- not show rewards / previous response to network? (meh)
- measure intra-task learning (last K trials - first K trials) rather than raw final perf? (meh)

---

Try again, full: 6 tasks, plasticity, alpha (initalpha .25). 500 gens, 200 trials (40 loss). Still .5x LR.

There MAY or may not be a small learning... The leaning slope is small but positive... But unsure (maybe yes. maybe no) for the withheld.

NOTE: Both runs had good dms performance, very bad withheld (dnms)... Is it coincidence or does the system "like" dms?...

Time total > 13000s, i.e. >3 hours and half !

What about 1 task per gen, but 2x as many gens? Try this now, 1 task per gen. 1010 gens. No obvious learning at all...


It really seems to like dms... is it because there's no anti-dms in the learned tasks, since anti-dmn (dnms) is the withheld?

Need more tasks!

---

200 gens, 200 trials per task, 1 task per gen. Always the same task ! Doesn't help...

Remove use of alpha for now... 500 gens, 500 trials.

It doesn't seem to be working! Maybe because I merfed response and reward signals? Shouldn't matter, only one task.

Wait... evolutionary loss initially decreases, but then increases a bit and plateaus.... Binary loss (reward) goes up, thenb crashes down within ~ 200 gens...?

Trying no plasticity, 20+20 trials, 190 gens.

BS was = 6, lol ! That might get weird gradients and confuse Adam....

Trying BS 300. No slowing! Works much better (without plast). 1.7s / gen. BS=500: 2.2s, BS=1300: 4.4s. Going with 500 for now.

Put in plasticity, all tasks ! Test on dnms. 200 trials / gen. Increase reward and response signals to 1.2. 

Doesn't really work, no progress, for either training or testing tasks. Well, I didn't put the plastic back in? Also, May need more than 1 task per gen, since the params were optimized for this....

One difficulty: There are multiple peaks to ~1.0 from the start! Not noise because nblosstrials is 100 ! The network just naturally knows how to perform the task...

Try again. 2 tasks per gen, 150 gen, LR .5x, still no plast (it's just to try): No work...

Try with only 2 tasks (not including the test one): yeah, now it's learning. But it plateaus.... Difficult to learn both tasks, though it does, a bit. Slow.


---


N=100, BS=1000, 200 trials (T4): 30s / gen.

N=70: 17s....

Same, BS=2000: 31s

Trying same as before BS=6 20K trials but with N=70. Still pretty good, maybe a little bit less than N=100.

Same, trying J=1.0 io 1.5 to see if it makes a difference. It does ! Very bad ! Extremely slow learning.

So why not try J=2.0 ? No big difference with 1.5.  

3.0: seems less good.

1.8, and separating the plastic weights into bpw (initialized at start of each task for now): OK.

Introducing an alpha - though of course for now it's just all ones, just to test if the infrastructure works: Seems OK.
Trying multiple gens per run !


Really need to think about different tasks within each batch, maybe taking antithetic into account...? Not sure.



---


AM .5 -> 1.0: Worse.
So AM back to .5.

In a Uniform distribution, Std Dev = (b - a) / sqrt(12), so here 2AlphaModul / sqrt(12). What if we use a Normal distribution of the same std dev? 

Trying this (AM = .5, STDEVMODUL = 2 * AM / sqrt(12)). A bit less good.

Now with AM =1 again (but Normal): Bad. Remove Normal. AM = .5.

ProbaModul = .3 io .1, AM = .25: Apparently a bit worse(cyan, bl4).

AM .5: Similar.

Trying again PM .1, AM .5: Confirmed, best.


---


With BS=6, time is ~independent of N. 70 to 200, same time. 
But with BS=1000, ~200 trials: N=100: 23s, N=200: 165s.

Note: 23s per (200 trials) gen => 200 gens in 1.27 hours (decimal, roughly one and a quarter)

So N down to 100. 

AM 4, ETA .03. Plateaus ~ .4.
ETA .1: Worse !
ETA .01: Better, keeps going up. Reaches ~.7, not plateauing, by 20K.

RewardSize = .3 i.o. 1.2 and RespSignalSize .3 i.o. =StimSize=3.1 (as before): Not much difference !

StimSize .5 io 3.1: Considerably worse...

Stimsize 5: Similar to StimSize 3 ! One gets perfect, most don't. 

Stimsize back to 3.1.  TAU 2.5 io 5. Not very different.

Tau back to 5. Eta .003. A bit worse.

ETA back .01. Only 1 resp: Much better of course, but far from perfect ! Doesn't even hit all-1 !

AlphaLossTrace .75. Maybe a bit worse, not much. .99: Also a little bit worse ! (again not much, may be noise)

AT back to .9. 
Autapses. Not much diff so remove.

AM 8: Worse!
AM 2: Way worse

AM .5, probamodul 0.03 io .003:  a bit worse but not too much

Same but eta .03 io .01: Best ! hits 1.0 (with dents) a bit after 15K trials, lol.

Eta .1: way worse. 

ETA .01, Probamodul .1 io .03: tiny bit less good than best, also touches 1.0.

ETA .03: Best by a margin! Start reaching 1.0 ~12000, consistent by ~15000 trials.

Evidently: many small perturbations are better than a few big ones.


---

OK, now that I've gotten single-gen, plasticity only to work on the DMS task within 20K trials/episodes... Time to make the task actually learning/memorizing - i.e. put the stimuli in succession i.o. simultaneous !

But first a bit of code cleanup.

succ stim nit very good, settles on all-2 often....

try with .3 diagonal. Maybe faster initially but same log term.

J=1.0: Same.

Back to J=1.5 and no special autapses, tau_et = 60 io 40. Doesn't help.

ProbaAnswer 1 .5 io 2/3.  Now both 0 and 2 resps, but perf is at ~.25 (as before).

Longer presentation : not much diff. 

Alphamodul 1 io 4, eta .3 io .1:  Worse!
Alphamodul 4, eta .3: Back to same as before.
Alphamodul 8, eta .1: Maybe a tiny bit better? (not sure)
AM 16, eta .05: Also a little bit better?
AM 16, eta .1: Nah.

Back to 4/.1,  N=200. Doesn't work bad, sometimes settle on the wrong answers.
1 / .03, N=200 still. Seems to work sometimes? There seems to be much instability... Keeps going up!
1/.01: Goes up more?
.5/.01: *very* bad. 
4/.01: Meh. Worse than the two goodish ones?
4/.003: Best so far, sometimes works at the end! (bl7, orange)
8/.001: Worse. (bl8, grey)
4/.001: Even a little bit worse (bl9, m)
8/.003: A bit below orange (second best then?); mostly comparable to grey.


---


ml1: maxdw 3e-3 io 1e-3. Worse initially, not clear (might be a tad better?)later.

ml3: probamodul .01 io .003: Initially faster, but similar later. More noisy.


Back to normal, recording both ml and bl: seems to be some variance, seems a bit slower than previous normal....

Trying it again, absoluteluy no change: lol, looks much worse !

The variance is simply too high, even with 7 runs, to get a useful assessment....

Try exact same here and in the second? bl1/2, ml1/2.

Now: Same, but mutiplying by meanerrtrace(trialtype) in the dw. With 14 runs all, seems slower than original, but late on no difference. 

Same as previous, but try  tanh i.o. l_relu (output neurons still sigmoid).

bl5: Same as previous, but stimsize 3 io 1 (rewardsize unchanged). Lol, works fully within ~10K trials (median), 5K trials (95 percentile)!

Same as previous but dms io watchstim2: much harder, does not succeed.

Same but stimsize back to 1.1: no change.

Stimsize 3, l_relu: meh, similar.

Back to tanh (stimsize still 3). Autapses not special. Not much change.

Same as previous, tau_ET 20 io 40: a bit worse?
Back to tau 40. ProbaModul / alphamodul .01/1.0 io .003/4.0. Much slower !

Alphamodul 8.0 (probamodul still .01): Like before.
8.0/ .003: Maybe worse later...?

4 / 0.003 / eta 0.01 io 0.003: again not much diff long term.

Several modif. Alphamodul 1, eta .001, Jinit 1.5 (io 1) alphalosstrace .75 io.9 and FIXED ! maxdw 3e-4 io 1e-3. Didnt work, so eta .003, maxdw 1e-3, jinit 1.1. Learning very bad, doesn't even get responses righht. alphamodul 4: at least it gets the response right but learning is still like before, plateaus at ~.4. Alphtrace .9: Same.

eta .001, maxdw 1e-4: just slower. Amodul 1.0, Jinit 1.5, eta 0.01: No...
Eta 0.1, maxdw 1e-3: Much better. Median of 14 runs reaches basically-1 by 20K trials.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import pdb

import scipy
from scipy import ndimage
from scipy import linalg

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import time

import numpy as np
from numpy import fft 

from scipy import io as spio
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.set_printoptions(precision=5) 
np.set_printoptions(precision=5) 



# Specify the test task (and its logical negation, which is also withheld from the training set)
# TESTTASK = 'nand'; TESTTASKNEG = 'and'
TESTTASK = 'dms'; TESTTASKNEG = 'dnms'


LR =  .3 * 1e-2  # Adam LR. 

NBRUNS = 1 # 5          # Number of runs
NBGEN = 1700 # 500      # Number of generations per run


N = 70  # Number of neurons in the RNN.


BS = 500 # 1000         # Batch size, i.e. population size for the evolutionary algorithm. 
assert BS % 2 == 0      # Should be even because of antithetic sampling.

# Same parameters as GR Yang:
TAU =  100  # Neuron membrane constant, in ms
DT = 20     # Duration of a timestep, in ms


# All the following times are in *timesteps*, not ms
T =  50      # Number of *timesteps* per trial
STIMTIME = 20       # Duration of stimulus input, total, *in timesteps* (not ms)
REWARDTIME = 10     # Duration of reward signal period
RESPONSETIME = 10   # Duration of responze period  
STARTRESPONSETIME = 25  # Timestep  at which response period starts
ENDRESPONSETIME = STARTRESPONSETIME + RESPONSETIME
STARTREWARDTIME = 36    # Timsestep at which reward is deliverd and reward signal starts
ENDREWARDTIME = STARTREWARDTIME + REWARDTIME
assert ENDREWARDTIME < T


JINIT = 1.5 #   Scale constant of initial network weights. See Section 2.7 in the MML paper.
TAU_ET = 1000.0    # Time constant of the eligibility trace (in ms)
PROBAMODUL = .1 #       Probability of receiving a random perturbation, for each neuron, at each timestep.
ALPHAMODUL =  .5 #      Scale of the random perturbations
ETA = .03 #             Learning rate for lifetime plasticity
MULOSSTRACE = .9    #   Time constant for the trace of previous losses that serves as a baseline for neuromodulation
MAXDW = 1e-2 #          Maximum delta-weight perimissible for lifetime plasticity (remember that plasticity is only applied once per trial, at reward delivery time)
INITALPHA = .5 #        Initial alpha (plasticity parameter) value


# The name of all the tasks
alltasks = ['01', 'anti01', '10', 'anti10', 'watchstim1', 'watchstim2' ,'dms',  'antiwatchstim2', 'antiwatchstim1', 'or', 'and', 'nor', 'nand', 'dnms']


NBRESPS = 1 # Only 1 possible response ("1") in addition to the null "no-response" ("0"), for now.


NBSTIMNEURONS = 2   # 2 Stimulus neurons. Stimuli are binary, so both neurons receive opposite-valued inputs (or 0)
NBREWARDNEURONS = 2 # 2 reward signal (correct/incorrect) neurons. Same as for stimulus neurons.
NBNOISENEURONS = 1  # Noise neurons (not used in this code)
NBBIASNEURONS = 1   # Bias neurons. Activations clamped to BIASVALUE.
NBINPUTNEURONS = NBSTIMNEURONS + NBREWARDNEURONS +  NBNOISENEURONS + NBBIASNEURONS    # The first NBINPUTS neurons in the network are neurons (includes the bias, noise and reward inputs)
NBRESPNEURONS = 1 + NBRESPS  # 0-resp plus all other resps (for now, only 1)
NBAUXNEURONS = 0 #  No Auxiliary neurons for now.
NBOUTPUTNEURONS = NBRESPNEURONS + NBAUXNEURONS      # The last NBOUTPUTNEURONS neurons in the network are output neurons
NBRESPSIGNALNEURONS = NBRESPNEURONS     # Neurons that receive the response signal ("what response did I just give?")
STIMNEURONS = np.arange(NBSTIMNEURONS)
INPUTNEURONS = np.arange(NBINPUTNEURONS)
OUTPUTNEURONS = np.arange(N-NBOUTPUTNEURONS, N)
RESPNEURONS = np.arange(N-NBOUTPUTNEURONS, N-NBOUTPUTNEURONS + NBRESPNEURONS)
assert N-NBOUTPUTNEURONS+NBRESPNEURONS == N-NBAUXNEURONS  # The last neurons are output neurons, and the last output neurons are aux neurons.
AUXNEURONS = np.arange(N-NBAUXNEURONS, N)   # Again, no auxiliary neurons in this code.
REWARDNEURONS = np.arange(NBSTIMNEURONS, NBSTIMNEURONS+NBREWARDNEURONS) # The neuron receiving (and broadcasting) the "success" signal is the one just after the stimulus inputs
NOISENEURONS = np.arange(NBSTIMNEURONS+NBREWARDNEURONS, NBSTIMNEURONS+NBREWARDNEURONS+NBNOISENEURONS)
BIASNEURONS = np.arange(NBSTIMNEURONS+NBREWARDNEURONS+NBNOISENEURONS, NBSTIMNEURONS+NBREWARDNEURONS+NBNOISENEURONS+NBBIASNEURONS)
FIRSTRESPSIGNALNEURON = NBSTIMNEURONS+NBREWARDNEURONS+NBNOISENEURONS+NBBIASNEURONS   # The first neuron that receives the response signal. We'll need this later
assert FIRSTRESPSIGNALNEURON == NBINPUTNEURONS
RESPSIGNALNEURONS = np.arange(FIRSTRESPSIGNALNEURON, FIRSTRESPSIGNALNEURON +NBRESPSIGNALNEURONS)

BIASVALUE = 1.0



NBTASKSPERGEN = 2 #  2 task blocks per generation


NBTRIALSLOSS = 100              # Evolutionary loss is evaluated over the last 100 trials of each block
NBTRIALS =  300 + NBTRIALSLOSS  # Total number of trials per block
# NBTRIALS =  4 + NBTRIALSLOSS 
NBNOISETRIALS = 0


WDECAY =  3e-4 # Evolutionary weight decay parameter (for the Adam optimizer)
L1PEN =  0 # No L1 penalty for now.
L1PENALPHA = 0 # Same for the alpha parameters
MUTATIONSIZE = .01 #  Std dev of the Gaussian mutations of the evolutionary algorithm

REWARDSIZE = 3.0 # Size of the binary reward signal (correct/incorrect)
STIMSIZE = 3.0 # Size of the stimulus input
RESPSIGNALSIZE = 3.0 # Size of the response signal
NOISESIZE = 0 # .2  # .1 # 0.1


# Generate all the inputs and targets of each trial, for all batch individuals, for a whole task block. That is, all the inputs and 
# targets of all trials are generated together for each block. We do that to ensure proper balancing.
def generateInputsAndTargetsForTask(mytask='watchstim1', whichresp=1):  # whichresp always 1 in this code, since NBRESPS = 1
    assert NBOUTPUTNEURONS == 1 + NBRESPS + NBAUXNEURONS
    assert whichresp in np.arange(1, NBRESPS+1)
    assert NBSTIMNEURONS == 2 
    assert mytask in alltasks
    assert NBTRIALS > NBTRIALSLOSS
    assert STIMTIME % 2 == 0


    returns=[]
    
    # We could just use random stimuli, but we want to balance the inputs and outputs.

    trials = []
    for trialpart in range(2): # We separately balance the trials that don't count for loss, and those that do. Then we concatenate at the end.
        nbt = NBTRIALSLOSS if  trialpart == 1 else (NBTRIALS - NBTRIALSLOSS)

        # Half inputs are 1, half are 0. Also, half identical b/w both stimuli, half different.
        stims1 = np.zeros((BS, nbt)).astype(int)
        stims2 = np.zeros((BS, nbt)).astype(int)
        stims1[:,:nbt//2] = 1
        stims2[:,::2] = 1

        # We need to shuffle independently in each row (i.e. for each batch element), but the same way for both stims1 and stims2 in each row
        # Taken from StackOverflow:
        p = np.argsort(np.random.rand(BS,nbt), axis=1)
        stims1 = (np.array(list(map(lambda x, y: y[x], p, stims1))))
        stims2 = (np.array(list(map(lambda x, y: y[x], p, stims2))))

        trials.append((stims1, stims2))

    # Now 'trials' is a list of 2 elements (non-loss trials and loss trials), each of which is a list of 2 elements (stims1, stims2), 
    # each of which is a long list of binary values (value of this particular stimulus for this particular trial).

    # We rearrange them into two numpy arrays: one for stims1 and one for stims2 (each including all trials, non-loss and loss)
    stims1 = np.concatenate((trials[0][0], trials[1][0]), axis=1)
    stims2 = np.concatenate((trials[0][1], trials[1][1]), axis=1)
    

    # Now we have two proper lists of binary value pairs, each giving the value of one stimulus input for each trial. Let's turn this 
    # into actual numerical inputs for the network:
    for numtrial in range(NBTRIALS):
        
        inpts = np.zeros((BS, NBSTIMNEURONS, STIMTIME)) # T)) # 
        
        StimDur = STIMTIME

        StartStim = 0

        # We're assuming that the first two neurons are stimulus neurons

        # The two stimuli are presented in succession, with both input neurons locked in opposite values to each other:
        inpts[:, 0, StartStim:StartStim+StimDur//2 - 2] = 2.0 * stims1[:, numtrial][:, None] - 1.0
        inpts[:, 0, StartStim+StimDur//2:StartStim+StimDur - 2] = 2.0 * stims2[:, numtrial][:, None] - 1.0
        inpts[:, 1, StartStim:StartStim+StimDur] = -inpts[:, 0, StartStim:StartStim+StimDur] 

    
        # Now we compute the targets, that is, the expected values of the output neurons, depending on inputs and tasks

        # The response is binary: either you respond (i.e. only the non-null response should be active), or you don't (only the null-response neuron should be active)

        # Note: For now, whichresp = NBRESPS = 1 - there is only one non-null response neuron (in addition to the null-response neuron).

        assert 1+whichresp >= 2; assert 1+whichresp < 2 + NBRESPS
        
        inpts = torch.from_numpy(inpts).float().to(device)


        # Generate the targets
        tgts = np.zeros((BS, NBRESPNEURONS, RESPONSETIME))

        # First we generate the expected output for the non-null response neuron, based on inputs and task:
        if mytask == 'watchstim1':
            tgts[:, whichresp, :] = stims1[:, numtrial][:, None]
        elif mytask == 'watchstim2':
            tgts[:, whichresp, :] = stims2[:, numtrial][:, None]
        elif mytask == 'antiwatchstim1':
            tgts[:, whichresp, :] = 1.0 - stims1[:, numtrial][:, None]
        elif mytask == 'antiwatchstim2':
            tgts[:, whichresp, :] = 1.0 - stims2[:, numtrial][:, None]
        elif mytask == 'and':
            tgts[:, whichresp, :] = (stims1[:, numtrial][:, None] * stims2[:, numtrial][:, None])
        elif mytask == 'nand':
            tgts[:, whichresp, :] = 1.0 - (stims1[:, numtrial][:, None] * stims2[:, numtrial][:, None])
        elif mytask == 'or':
            tgts[:, whichresp, :] = np.clip(stims1[:, numtrial][:, None] + stims2[:, numtrial][:, None], 0.0, 1.0)
        elif mytask == 'nor':
            tgts[:, whichresp, :] = 1.0 - np.clip(stims1[:, numtrial][:, None] + stims2[:, numtrial][:, None], 0.0, 1.0)
        elif mytask == '10':
            tgts[:, whichresp, :] = stims1[:, numtrial][:, None] * (1.0 - stims2[:, numtrial][:, None])
        elif mytask == 'anti10':
            tgts[:, whichresp, :] = stims1[:, numtrial][:, None] * (1.0 - stims2[:, numtrial][:, None])
        elif mytask == '01':
            tgts[:, whichresp, :] = (1.0 - stims1[:, numtrial][:, None]) * stims2[:, numtrial][:, None]
        elif mytask == 'anti01':
            tgts[:, whichresp, :] = 1.0 - (1.0 - stims1[:, numtrial][:, None]) * stims2[:, numtrial][:, None]
        elif mytask == 'dms':
            tgts[:, whichresp, :] = (stims1[:, numtrial][:, None] == stims2[:, numtrial][:, None])
        elif mytask == 'dnms':
            tgts[:, whichresp, :] = (stims1[:, numtrial][:, None] != stims2[:, numtrial][:, None])
        else:
            raise ValueError("Task not in list of allowed tasks! Task is"+str(mytask))

        assert np.all(np.logical_or(tgts == 0.0 , tgts == 1.0))

        # The null-response neuron's expected output is just the opposite of the non-null response neuron output (either you respond, or you don't !)
        tgts[:, 0, :] = 1.0 - tgts[:, whichresp, :]

        tgts = torch.from_numpy(tgts).float().to(device)     

        # In practice, we clip targets to 0.1/0.9 instead of actually 0.0/1.0. This may or may not help.
        tgts.clip_(min=0.1, max=0.9)



        # Finally: antithetic pairs get  identical sequences of trials !
        inpts[BS//2:, :, :] = inpts[:BS//2, :, :]
        tgts[BS//2:, :, :] = tgts[:BS//2, :, :]




        returns.append((inpts, tgts))
    
    return returns


totalnbtasks = 0
ticstart = time.time()
with torch.no_grad():

    # traincurves=[]
    evolosses_allruns=[]
    evolossesaux_allruns=[]
    evolossesconc_allruns=[]
    binarylosses_allruns=[]

    ml0sallruns = []
    bl0sallruns = []
    
    PRINTING = True # if numgen == 0 or np.random.rand() < .05 else False


    # The code allows to have multiple runs in succession, all totally independent from each other. 
    # In practice, this is useful if you don't have too many generations per run (as with the NAND test task). 
    # For the DMS test task, we only use one run because it requires so many generations that multiple runs in the same session 
    
    for numrun in range(NBRUNS):

        # Initialize innate weights values
        w =  torch.randn(N,N)  * JINIT / np.sqrt(N) 
        w = w.to(device)
        
        # Initialize alpha values - the plasticity parameters (capital-pi in the paper)
        alpha = INITALPHA * torch.ones_like(w).to(device)

        # We zero out input weights to input neurons, though it probably doesn't have any effect.
        w.data[:NBINPUTNEURONS, :] = 0   # Each *row* of w contains the weights to a single neuron.
        winit = w.clone()

        # We will be using the Adam optimizer to apply our (hand-computed) evolutionary gradients
        optimizer = optim.Adam([w, alpha], lr=LR, weight_decay=WDECAY)  # Default betas=(0.9, 0.999)

        # Evolosses are real-valued losses used for evolution. Binarylosses are binary 'correct/wrong' signals, also used for logging.
        evolosses = []
        responses0 = []
        binarylosses = []
        wgradnorms = []
        mytaskprev = mytaskprevprev = mytaskprevprevprev = whichrespprev = whichrespprevprev = whichrespprevprevprev = -1
        

        # Ready to start the evolutionary loop, iterating over generations (i.e. lifetimes). 

        for numgen in range(NBGEN):

            # Every 10th generation is for testing on the withheld task (with no weight change)
            TESTING = False
            if numgen == 0 or numgen == NBGEN-1 or numgen % 10 == 0:
                TESTING = True
                if PRINTING:
                    print("TESTING")


            tic = time.time()   
            responses0thisgen = []
        
            # Generating the population of mutated individuals:

            # First, batch the weights.
            bw = torch.dstack(BS*[w]).movedim(2,0).to(device)     # batched weights
            balpha = torch.dstack(BS*[alpha]).movedim(2,0).to(device)     # batched alphas
            # Generate the mutations, for both w and alpha
            # NOTE: batch element 0 (and BS/2, its antithetic pair) are NOT mutated, represent the curent unmutated candidate genotype.
            mutations_wandalpha = []
            for  n, x in enumerate( (bw, balpha) ):
                mutations = torch.randn_like(x, requires_grad=False).to(device) *  MUTATIONSIZE
                mutations[0,:,:] = 0  # 1st item in batch = current candidate
                mutations[BS//2:, :, :] = -mutations[:BS//2, :, :]    # Antithetic sampling for mutations ! Really helps.
                if TESTING:
                    mutations *= 0.0  # Not strictly necessary (no weight change is applied) - results in batch score variance being caused only by randomness in trial order
                x += mutations  
                mutations_wandalpha.append(mutations)


            
            bw.data[:, :NBINPUTNEURONS, :] = 0  # Input neurons receive 0 connections. Probably not necessary.
            bworig = bw.clone()                 # Storing the weights for comparison purposes at the gradient step (below).

            responsecounts = torch.zeros(BS, NBRESPS + 1).to(device)
            lifelosses = torch.zeros(BS, requires_grad=False).to(device)
            lifemselosses = torch.zeros(BS, requires_grad=False).to(device)
            # lifemselossesaux = torch.zeros(BS, requires_grad=False).to(device)
            lifeblosses = torch.zeros(BS, requires_grad=False).to(device)
                
            allresps=[]


            # Lifetime loop, iterates over task-blocks:
            for numtask in range(NBTASKSPERGEN):
                totalnbtasks += 1

                # bpw = batched plastic weights
                bpw = torch.zeros_like(bw).to(device)  # For now, plastic weights are initialized to 0 at the beginning of each task.

                # Initialize neural states
                bstates = .1 * torch.ones(BS, N).to(device)  # bstates (batched states) contains the neural activations (before nonlinearity). Dimensionality appropriate for batched matrix multiplication. 
                bstates[:, INPUTNEURONS] = 0
                bresps = 1.0 * bstates  # bresps is the actual neural responses, after nonlinearity, and also serve as the input for the next step.
                bresps[:, BIASNEURONS] = BIASVALUE

                meanlosstrace = torch.zeros(BS, 2 * 2).to(device)
                bls = []    # Will store binary losses of all batch elements, for each trial of this task
                bl0s = []   # Same but only for batch element 0 (i.e. the unmutated candidate genome)
                ml0s = []   # MSE loss 


                # Choose the task ! If not testing, makes sure it's different from recently chosen tasks.

                if TESTING: 
                    mytask = TESTTASK 
                    mytasknum = alltasks.index(mytask)
                    whichresp = 1   # This is always 1 for now, because NBRESP=1, i.e. we only use one response output (other than the null, "no-response" output)
                else:
                    while True:
                        mytasknum = np.random.randint(len(alltasks))

                        mytask = alltasks[mytasknum]
                        whichresp = 1 

                        if ( (mytask!= TESTTASK)  
                            and (mytask != TESTTASKNEG)  # We withhold both the test task and its logical negation
                            and     (mytask != mytaskprev) 
                                and (mytask != mytaskprevprev) 
                        ):

                            break

                    mytaskprevprev = mytaskprev; mytaskprev= mytask
                    whichrespprevprev = whichrespprev; whichrespprev = whichresp
                
                # Cumulative MSE and binary losses for this task, over the last NBLOSSTRIALS of the block:
                taskmselosses = torch.zeros_like(lifemselosses).to(device)
                taskblosses = torch.zeros_like(lifemselosses).to(device)

                if PRINTING:
                    print("task", mytask, "whichresp:", whichresp)
                
                # OK, ready to start the task.

                # Generate the task data (inputs and targets) for all trials:
                taskdata = generateInputsAndTargetsForTask(mytask=mytask, whichresp=whichresp)

                eligtraces =   torch.zeros_like(bw, requires_grad=False).to(device)  # Initialize the eligibility traces at the start of each block/task.

                # Task loop, iterating over trials
                # You do NOT erase memory (neural activations or plastic weights) between successive trials ! 
                for numtrial in range(NBTRIALS):
                    
                    # Initializations
                    losses = torch.zeros(BS, requires_grad=False).to(device)     # Losses for this trial
                    mselossesthistrial = torch.zeros(BS, requires_grad=False).to(device)     # Losses for this trial
                    totalouts = torch.zeros(BS, NBOUTPUTNEURONS, requires_grad=False).to(device)  

                    # Extract the inputs and targets for this trial:
                    inputs, targets = taskdata[numtrial]
                    assert inputs.shape == (BS, NBSTIMNEURONS, STIMTIME)

                    # Run the network. Trial loop, iterating over timesteps
                    for numstep in range(T):          

                        # Update neural activations
                        bstates += (DT / TAU) * (-bstates +  torch.bmm((bw + balpha * bpw), bresps[:, :, None])[:,:,0] )  


                        # Applying the random perturbations on neural activations, a.k.a. "modulations", both for noise and for the lifetime plasticity algorithm (node-perturbation)
                        # Ans also updating the eligibility trace appropriately
                        if numstep > 1 : 
                            modulindices =  (torch.rand(1, N) < PROBAMODUL).int()   # Which neurons get perturbed?
                            modulations = (ALPHAMODUL * modulindices * (2 * torch.rand(1, N) - 1.0)).to(device)  # Note the dimensions: the same noise vector is applied to all elements in the batch (to save time!)
                           
                            bstates += modulations
                            
                            # Node-perturbation: Hebbian eligibility trace = product between inputs (bresps from previous time step) and *perturbations* in outputs. dH = X * deltaY 
                            # We do this with a (batched) outer product between the (column) vector of modulations (1 per neuron) and the (row) vector of inputs
                            # Note that here, since we have an RNN, the input is bresps - the network's responses from the previous time step
                            if torch.sum(modulindices) > 0:
                                eligtraces += torch.bmm( modulations.expand(BS, -1)[:, :, None],  bresps[:, None, :] ) 

                        # Eligibility traces, unlike actual plastic weights, are decaying
                        eligtraces -=  (DT / TAU_ET) * eligtraces


                        # We can now compute the actual neural responses for this time step, applying the appropriate nonlinearity to each neuron
                        bresps = bstates.clone() # F.leaky_relu(bstates)
                        # The following assumes that output neurons are the last neurons of the network !                        
                        bresps[:,N-NBOUTPUTNEURONS:].sigmoid_()     # The output neurons are sigmoids, all others are tanh. An arbitrary design choice.
                        bresps[:,:N-NBOUTPUTNEURONS].tanh_()
                        

                        torch.clip_(bresps, min=-5, max=5)  # Unnecessary if all nonlinearities are bounded.


                        # Are we in the input presentation period? Then apply the inputs.
                        # Inputs are clamping, fixing the response of the input neurons.
                        if numstep < STIMTIME:
                            bresps[:, STIMNEURONS] = STIMSIZE * inputs[:, :, numstep]        
                        else:
                            bresps[:, STIMNEURONS] = 0

                        bresps[:, BIASNEURONS] = BIASVALUE


                        # Are we in the response period? Then collect network response.
                        if numstep >= STARTRESPONSETIME and numstep < ENDRESPONSETIME:

                            assert numstep < STARTREWARDTIME
                            # Accumulate the total activation of each output neuron, so that we can compute the network's actual response at the end of response period:
                            totalouts +=  bresps[:, OUTPUTNEURONS] 
                            # Accumulate the MSE error between actual and expected outputs:
                            mselossesthistrial += torch.sum( (bresps[:, RESPNEURONS] - targets[:, :, numstep - STARTRESPONSETIME]) ** 2, axis=1 ) / RESPONSETIME

                        else:
                            bresps[:, OUTPUTNEURONS] = 0.0


                        # Is the response period finished, or equivalently, are we at the first step of the reward / feedback period?
                        # If so, compute the network's response and apply plasticity (i.e. deliver the neuromodulation).
                        if numstep == STARTREWARDTIME:
                            # The network's response for this trial (0 or 1) is the index of the output neuron that had the highest cumulative output over the response period
                            totalresps = totalouts[:, :NBRESPNEURONS]   # Assumes that the response neurons are the first among output neurons !
                            responses = torch.argmax(totalresps, dim=1)  # responses is a 1D, integer-valued array of size BS. totalresps is a 2D real-vlued array of size BS, NBRESPS+1                           
                            
                            # blosses (binary losses) is a 1/-1 "correct/wrong" signal for each batch element for this trial.
                            blosses = 2.0 * (responses == torch.argmax(targets[:, :, 0], dim=1)).float() - 1.0    
                            responses0thisgen.append(float(responses[0]))

                            # We also want the 1-hot version of the response for each neuron. This will be used as the response signal below.
                            if numtrial > 0:
                                responses1hot_prev = responses1hot.clone()
                            responses1hot = F.one_hot(responses, 1+NBRESPS)
                            if numtrial < NBTRIALS - NBTRIALSLOSS:
                                responsecounts += responses1hot

                            # Now we apply lifetime plasticity, with node-perturbation, based on eligibility trace and suitably baselined reward/loss

                            # We compute separate baseline (running average) losses for different types of trials, as defined by their inputs (as in Miconi, eLife 2017). 
                            # So we need to find out the trial type for each element in batch.
                            input1 = inputs[:, 0, 0]; input2 = inputs[:, 1, 0]  # This assumes that inputs are constant and start from timestep 0 !
                            trialtypes = (input1>0).long() * 2 + (input2>0).long()

                            # Compute and apply the plasticity, based on accumulated eligibility traces and reward                          
                            if numtrial > 30:  # Lifetime plasticity is only applied after a few burn-in trials.
                                # eligtraces: BS x N x N.   mselossesthhistrial:  BS.    meanlosstrace: BS x (N.N).    trialtypes: BS
                                # dw should have shape BS x N x N, i.e. one for each connection and batch element. Do not sum over batch dimension! The batch is purely evolutionary !


                                dw =   (ETA * eligtraces  * (  meanlosstrace[np.arange(BS), trialtypes] * (mselossesthistrial - meanlosstrace[np.arange(BS), trialtypes]) )[:, None, None]).clamp(-MAXDW, MAXDW)
                                # dw =   (ETA * eligtraces  * (  meanlosstrace[np.arange(BS), trialtypes] * (mselossesthistrial ) )[:, None, None]).clamp(-MAXDW, MAXDW)


                                bpw -= dw

                                if PRINTING and np.random.rand() < .02:
                                    print(numrun, numtrial, "{:.4f}".format(float(torch.mean(mselossesthistrial))), "{:.4f}".format(float(torch.norm(bw+bpw))), 
                                                                        "{:.4f}".format(float(torch.norm(dw))), "{:.4f}".format(float(torch.norm(bpw))))
                            # Updating the baseline - running average of losses, for each batch element, for the trial type just seen
                            meanlosstrace[torch.arange(BS).long(), trialtypes] *= MULOSSTRACE
                            meanlosstrace[torch.arange(BS).long(), trialtypes] +=  (1.0 - MULOSSTRACE) * mselossesthistrial



                        # Are we in the reward signal period?
                        # The neuromodulatory reward signal is applied just once per trial, above. Here we provide a binary "correct/ incorrect" signal to the network, 
                        # i.e. "was my response right or wrong for this trial?" 
                        # We also provide a signal indicating which response it gave in this trial (in theory it should be able to calculate it itself if needed, but this may help)
                        if numstep >= STARTREWARDTIME and numstep < ENDREWARDTIME: # Note that by this time, the loss has been computed and is fixed
                            
                            # We provide a binary, "correct/incorrect" signal to the network
                            bresps[:,REWARDNEURONS[0]] = REWARDSIZE * blosses[:]         # Reward input is also clamping
                            bresps[:,REWARDNEURONS[1]] = -REWARDSIZE * blosses[:]         # Reward input is also clamping

                            bresps[:,REWARDNEURONS].clip_(min=0)

                            # We provide the network with a signal indicating the actual response it chose for this trial.
                            # bresps[:, RESPSIGNALNEURONS] = responses1hot.float()
                            # # The following is probably unnecessarily obscure and may be replaced with simply putting in the one-hot version of responses multiplied by Rewardsize:
                            bresps[:, RESPSIGNALNEURONS] = 0
                            bresps.scatter_(1, FIRSTRESPSIGNALNEURON + responses[:, None],   RESPSIGNALSIZE)  # The ResponseSignalNeuron corresponding to the actual response gets value RESPSIGNALSIZE - all others get 0

                            
                        else:
                            bresps[:,REWARDNEURONS] = 0
                            bresps[:, RESPSIGNALNEURONS] = 0


                   
                    # Now all steps done for this trial:
                
                    if PRINTING:
                        if np.random.rand() < .1: 
                            print("|", int(responses[0]), int(blosses[0]), end=' ')
                    
                    ml0s.append(float(mselossesthistrial[0]))
                    bl0s.append(float(blosses[0]))
                    bls.append(blosses.cpu().numpy())
 

                    # If this trial is part of the last NBTRIALSLOSS, we accumulate its trial loss into the agent's total loss for this task.
                    if numtrial >= NBTRIALS - NBTRIALSLOSS:     # Lifetime  losses are only estimated over the last few trials
                        taskmselosses += 2 * mselossesthistrial / NBTRIALSLOSS   # the 2* doesn't mean anything
                        taskblosses += blosses / NBTRIALSLOSS
                

                # Now all trials done for this task:
                if PRINTING:
                    print("Med task mseloss:", "{:.4f}".format(float(torch.median(taskmselosses))))


                    # print("")
                lifemselosses += taskmselosses / NBTASKSPERGEN 
                lifeblosses += taskblosses / NBTASKSPERGEN 
            
                if (TESTING or numgen == 0) and numtask == 0:
                    # These files contain respectively the first and *latest* Testing block of the *current* run only. 
                    FNAME = 'bl1block_gen0.txt' if numgen == 0 else 'bl1block_lastgen.txt'
                    # np.savetxt(FNAME, np.array(bl0s))
                    np.savetxt(FNAME, np.vstack(bls))
                if numtask == 0 and (numgen == 0 or numgen == NBGEN-1):
                    # We append the 1st and last block of each run, all together alternating in the same array
                    assert (TESTING)
                    ml0sallruns.append(ml0s)
                    bl0sallruns.append(bl0s)


            # After all tasks done for this lifetime / generation:

            # if (numgen+1) == NBGEN // 3 or (numgen+1) == 2 * NBGEN // 3:
            #     for g in optimizer.param_groups:
            #         # g['lr'] /= 4.0 
            #         g['lr'] /= 10.0 
            #         g['weight_decay'] /= 10.0


            # responseprobas = responsecounts /  torch.sum(responsecounts, dim=1, keepdim=True)
            # responseprobassq = responseprobas * responseprobas
            # concentrations = torch.sum(responseprobassq, dim=1)

            lifelosses = lifemselosses 

            binarylosses.append(float(lifeblosses[0]))
            evolosses.append(float(lifemselosses[0]))

            np.savetxt('blosses_onerun.txt', np.array(binarylosses))
            np.savetxt('mselosses_onerun.txt', np.array(evolosses))
            

            # Now we're ready to perform evolution (by computing gradients by hand, and then applying the optimizer with these gradients)
            optimizer.zero_grad()

            # Gradient is just loss x mutation (remember we use antithetic sampling)
            gradient = torch.sum(mutations_wandalpha[0] * lifelosses[:, None, None], axis=0) # / BS
            wgradnorm = float(torch.norm(gradient))
            wgradnorms.append(wgradnorm)
            if PRINTING:
                print("norm w:", "{:.4f}".format(float(torch.norm(w))), "norm gradient:", "{:.4f}".format(wgradnorm), 
                        "norm a:", "{:.4f}".format(float(torch.norm(alpha))), "mean a:",  "{:.4f}".format(float(torch.mean(alpha))))


            w.grad = gradient
            wprev = w.clone()

            gradientalpha = torch.sum(mutations_wandalpha[1] * lifelosses[:, None, None], axis=0) # / BS
            alpha.grad = gradientalpha
            alphaprev = alpha.clone()

            if numgen > 0 and not TESTING:
                optimizer.step()
                w -= L1PEN * torch.sign(w)
                alpha -= L1PENALPHA * torch.sign(alpha)
            
            wdiff = w - wprev
            adiff = alpha - alphaprev
            if PRINTING:
                print("Norm w-wprev:", "{:.4f}".format(float(torch.norm(wdiff))), "Max abs w-wprev:", "{:.4f}".format(float(torch.max(torch.abs(wdiff)))), 
                    "Norm a-aprev:", "{:.4f}".format(float(torch.norm(adiff))), "Max abs a-aprev:", "{:.4f}".format(float(torch.max(torch.abs(adiff))))  )

        

            if PRINTING:
                print("Med/min/max/Half-Nth/0th loss in batch:", float(torch.median(lifelosses)), float(torch.min(lifelosses)), float(torch.max(lifelosses)),
                                        float(lifelosses[BS//2]), float(lifelosses[0]))
                print("Gen", numgen, "of run", numrun, "done in", time.time()-tic)
        


        # After all generations for this run are done:

        # These arrays contain the first and last block (both on test task) of each run to date, in alternation
        mlnp = np.vstack(ml0sallruns)
        np.savetxt('meanlosses0_1stLastB_allruns.txt', mlnp)
        blnp = np.vstack(bl0sallruns)
        np.savetxt('blosses0_1stLastB_allruns.txt', blnp)

        binarylosses_allruns.append(binarylosses)
        evolosses_allruns.append(evolosses)

        with open('binlossDMS_LR'+str(LR)+'MutSize'+str(MUTATIONSIZE)+'NoiseS'+str(NOISESIZE)+'_RSize'+str(REWARDSIZE)+'SSize'+str(STIMSIZE)+'.txt', 'w') as myfile:
            for c in binarylosses_allruns:
                for l in c:
                        myfile.write(str(l)+" ")
                myfile.write("\n")
        with open('evolossDMS_LR'+str(LR)+'MutSize'+str(MUTATIONSIZE)+'NoiseS'+str(NOISESIZE)+'_RSize'+str(REWARDSIZE)+'SSize'+str(STIMSIZE)+'.txt', 'w') as myfile:
            for c in evolosses_allruns:
                for l in c:
                        myfile.write(str(l)+" ")
                myfile.write("\n")



print("Time taken:", time.time()-ticstart)

In [ ]:
import matplotlib.pyplot as plt; import numpy as np

# Showing the success rate over the batch (i.e. the mean binary loss over the batch) for each trial of the last test-task block.

z = .5 + .5 * np.vstack(bls)
print(z.shape)
plt.plot(np.mean(z, axis=1))

In [ ]:
# % correct over last 100 trials of each generation (unmutated candidate genotype only), shown separately for the non-test tasks (blue) and test task (red)
# Median and IQR over all runs. 

bl = .5 + .5 * np.loadtxt('binlossDMS_LR0.003MutSize0.01NoiseS0_RSize3.0SSize3.0.txt')
if(len(bl.shape)<2):   # If there is only a single run, add a singleton dimension
    bl = bl[None, :]
print(bl.shape)
ss = bl.shape[1]  # Number of generations

plt.figure(figsize=(4,4))

xr = np.arange(len(bl[0,:]))
plt.fill_between(xr[xr%10 != 0], np.quantile(bl, .25, axis=0).T[xr % 10 != 0], np.quantile(bl, .75, axis=0).T[xr % 10 != 0], color='b', alpha=.3)
plt.plot(xr[xr % 10 != 0], np.quantile(bl, .5, axis=0).T[xr % 10 != 0], 'b', label='Training tasks');
plt.fill_between(xr[::10], np.quantile(bl, .25, axis=0).T[0::10], np.quantile(bl, .75, axis=0).T[0::10], color='r', alpha=.3)
plt.plot(xr[::10], np.quantile(bl, .5, axis=0).T[0::10], 'r', label='Test task')


plt.xlabel('Generations')
plt.ylabel('% correct over last 100 trials')
plt.legend(loc='lower right')

plt.title('Test task: '+str(TESTTASK).upper())

plt.tight_layout()

plt.savefig('evoloss'+str(TESTTASK).upper()+'.png', dpi=300)


In [ ]:
# % correct over all *runs* (unmutated candidate genotype only), for the first and last block (both on the test task).

from matplotlib.ticker import FormatStrFormatter

bla = np.loadtxt('blosses0_1stLastB_allruns.txt')
print(bla.shape)
nbruns = bla.shape[0]//2

plt.subplot(2,1,1);
mm = np.mean(bla[0::2, :]>0, axis=0)
plt.plot(mm, 'g')
plt.plot(np.arange(mm.size), .5*np.ones_like(mm), 'k:')
plt.ylim([-.1, 1.1])
#mm = np.mean(bla[1::2, ::2]>0, axis=0)
#plt.plot(np.arange(0,400,2), mm, 'g')
#plt.plot(smooth(mm), 'k')
plt.ylabel("% Correct (over "+str(nbruns)+" runs)")
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
plt.twinx()
plt.gca().get_yaxis().set_ticks([])
plt.ylabel("Generation 0")

plt.subplot(2,1,2);
mm = np.mean(bla[1::2, :]>0, axis=0)
plt.plot(mm, 'g')
plt.plot(np.arange(mm.size), .5*np.ones_like(mm), 'k:')
plt.ylim([-.1, 1.1])
#mm = np.mean(bla[1::2, ::2]>0, axis=0)
#plt.plot(np.arange(0,400,2), mm, 'g')
#plt.plot(smooth(mm), 'k')
plt.ylabel("% Correct (over "+str(nbruns)+" runs)")
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
plt.xlabel("Trials")
plt.twinx()
plt.gca().get_yaxis().set_ticks([])
plt.ylabel("Generation "+str(ss))

plt.tight_layout()
plt.savefig('trials'+str(TESTTASK).upper()+'.png', dpi=300)

In [ ]:
print("Med/min/max/Half-Nth/0th loss in batch:", float(torch.median(lifelosses)), float(torch.min(lifelosses)), float(torch.max(lifelosses)),
      float(lifelosses[BS//2]), float(lifelosses[0]))


In [ ]:
print(inputs.shape, targets.shape)

print(mytask)
for nn in range(6):
    print("Trial", nn)
    ii, tt = taskdata[nn]
    print(ii[0,0, 0::15])  # Stim Input 0 for this trial, timesteps 0 and 15, batch element 0
    print(tt[0, :, 0])  # Targets for this trial, timestep 0,  batch element 0

for nn in range(6):
    print("Trial", nn)
    ii, tt = taskdata[nn]
    print(ii[1,0, 0::15])  # Stim Input 0 for this trial, timesteps 0 and 15, batch element 0
    print(tt[1, :, 0])  # Targets for this trial, timestep 0,  batch element 0


In [ ]:
# This was code for showing the trajectories of multiple neurons over several trials, but we don't store them ('allresps') any more due to mempry constraints

if False:
    ar1 = torch.dstack(allresps) # Now allresps has dimensions batchsize x neurons x time

    # If the network is not chaotic enough and they all have the same weights, they will produce very similar final outputs independently of initial conditions - because eigenvectors !
    ar = ar1.cpu().numpy()
    print(ar.shape)
    ar[ar>1.5] = 1.5
    ar[ar<-1.5] = -1.5

    plt.figure(figsize=(20,20))
    z = plt.plot(ar[1,:20,-1000:].T)  # 20 first neurons of batch element 1
    z = plt.plot(ar[1,-1,-1000:].T, 'r--')  # Last (response) neuron of batch element 1
    z = plt.plot(ar[1,-2,-1000:].T, 'b--')  # Last (response) neuron of batch element 1
    plt.figure(figsize=(10,10))
    z = plt.plot(ar[0,:20,-1000:].T)  # 20 first neurons of batch element 0
    z = plt.plot(ar[0,-1,-1000:].T, 'r--')  # Last (response) neuron of batch element 0
    z = plt.plot(ar[0,-2,-1000:].T, 'b--')  # Last (response) neuron of batch element 0


In [ ]:
# Show the final evolved w matrix (innate weights), and its column and row averages

import matplotlib.pyplot as plt; import numpy as np

ww = w.cpu().numpy()
print("Std W:", np.std(ww), "Norm W", np.sqrt(np.sum(ww**2)))
zz = np.random.randn(N,N)  * JINIT / np.sqrt(N) 
print("Std w_init_like:", np.std(zz), "norm:", np.sqrt(np.sum(zz**2)))
print(np.min(ww), np.max(ww), np.median(ww))
print(np.min(zz), np.max(zz), np.median(zz))

plt.matshow(ww)
plt.figure()
plt.plot(np.mean(ww, axis=0), 'r')
plt.figure()
plt.plot(np.mean(ww, axis=1), 'b')

In [ ]:
# Show the final evolved alpha matrix (plasticity parameters, capital-Pi in the MML paper)


aa = alpha.cpu().numpy()

print(np.mean(aa))
print(np.min(aa), np.max(aa), np.median(aa))
plt.matshow(aa)
plt.figure()
plt.plot(np.mean(aa, axis=0), 'r')
plt.figure()
plt.plot(np.mean(aa, axis=1), 'b')

In [ ]:
# Show the mean plastic weights over the whole batch., and for two elements in the batch (note that this is after the end of the last block)

pw = bpw.cpu().numpy()

print(pw.shape)
plt.matshow(np.mean(pw, axis=0))
print(np.mean(np.abs(w.cpu().numpy())), np.mean(np.abs(pw)))
print(np.max(np.abs(w.cpu().numpy())), np.max(np.abs(pw)))
print(np.min(pw[0,:,:]), np.max(pw[0,:,:]), np.median(pw[0,:,:]))
plt.figure()
plt.matshow(pw[0,:,:])
plt.figure()
plt.matshow(pw[1,:,:])


In [ ]:
ww = w.cpu().numpy()
pw0 = bpw[0,:,:].cpu().numpy()
aa = alpha.cpu().numpy()
np.savetxt('w.txt', ww)
np.savetxt('pw0.txt', pw0)
np.savetxt('alpha.txt', aa)